# Building up

In [1]:
import numpy as np
import time
time_start = time.time()
X = np.genfromtxt('00_Data/train_X_dog_cat.csv', delimiter=',')
y = np.genfromtxt('00_Data/train_y_dog_cat.csv', delimiter=',')
print(X.shape)

n = X.shape[0]
d = X.shape[1]
print(y.shape)
time_end = time.time()
print("Done loading training data, using time:(min)", (time_end-time_start)/60)

(1953, 3072)
(1953,)
Done loading training data, using time:(min) 0.06361965735753378


In [2]:
X_test = np.genfromtxt('00_Data/test_X_dog_cat.csv', delimiter=',')
y_test = np.genfromtxt('00_Data/test_y_dog_cat.csv', delimiter=',')
m = X_test.shape[0]
print(m)

2000


In [3]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler().fit(X)
X = scaler.transform(X)
X_test = scaler.transform(X_test)


In [9]:
def linear_kernel(x1,x2):
    # x1 = (n,d) X2 = (m,d)
    return(np.dot(x1,x2.T))

def inhomogeneous_poly_kernel(x1,x2):
    n = x1.shape[0]
    m = x2.shape[0]
    return np.power(np.ones(shape=(n,m)) + np.dot(x1,x2.T), 5)

                  
def gaussian_kernel(x1,x2,sigma=100):
    n = x1.shape[0]
    m = x2.shape[0]
    K = np.zeros(shape=(n,m))
    for i in range(n):
        for j in range(m):
            K[i][j] = gaussian(x1[i],x2[j],sigma)
    return (K)

def gaussian(x,y,sigma):
    return np.exp((-(np.linalg.norm(x-y,2)**2))/(2*sigma**2))

In [5]:
def evaluation(ans, y):
    right = 0
    num = ans.shape[0]
    for i in range(ans.shape[0]):
        if (ans[i] - 0.5)*y[i] > 0:
            right += 1
    return(right/float(num))

In [6]:
def tolerable(delta,tol):
    from numpy import linalg as LA
    if LA.norm(delta,1) > tol:
        return False
    else:
        return True

In [26]:
def probability(alpha, Ki):
    s = np.dot(alpha, Ki)
    return(1/(1+np.exp(-s)))

def train(K,y,max_iter,batch_size,regu_para, eta,tol):
    time_start = time.time()
    #alpha = np.random.uniform(size=(n))
    alpha = np.zeros(n)
    for t in range(max_iter):
        samples = np.random.permutation(n)[:batch_size]
        g = regu_para * alpha
        #g = np.zeros(n)
        for i in samples:
            pi = probability(alpha, K[i])
            g[i] = g[i] + (pi - (y[i]+1)/2.0)
        alpha = alpha - eta*g
        if tolerable(g, tol):
            break
        if t % 2000 == 0:
            print("iter %d train_accuracy:%f"%(t,evaluation(probability(alpha,K), y)))
    time_end = time.time()
    print("Done training, using time:(min)", 
          (time_end-time_start)/60)
    return(alpha)

In [27]:
def experiment(kernel, X,y,X_test,y_test,regu_para, eta,tol= 0.1, max_iter=50000, batch_size=640):
    K = kernel(X,X)
    K_test = kernel(X,X_test)
    print("Done kernel generating")
    alpha = train(K,y, max_iter,batch_size,regu_para, eta,tol)
    print("Done training")
    print(alpha)
    print("training result")
    ans = probability(alpha,K)
    #print(ans)
    print(evaluation(ans, y))
    #print(y)
    print("test result")
    ans = probability(alpha,K_test)
    #print(ans)
    print(evaluation(ans, y_test))
    #print(y_test)

In [28]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=0, eta=0.01, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.520225


/home/y247xie/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


iter 2000 train_accuracy:0.555044
iter 4000 train_accuracy:0.589350
iter 6000 train_accuracy:0.593446
iter 8000 train_accuracy:0.624680
iter 10000 train_accuracy:0.665643
iter 12000 train_accuracy:0.629288
iter 14000 train_accuracy:0.621608
iter 16000 train_accuracy:0.697901
iter 18000 train_accuracy:0.627240
iter 20000 train_accuracy:0.720430
iter 22000 train_accuracy:0.710701
iter 24000 train_accuracy:0.661546
iter 26000 train_accuracy:0.632873
iter 28000 train_accuracy:0.604199
iter 30000 train_accuracy:0.584229
iter 32000 train_accuracy:0.654378
iter 34000 train_accuracy:0.636457
iter 36000 train_accuracy:0.675371
iter 38000 train_accuracy:0.584741
iter 40000 train_accuracy:0.719406
iter 42000 train_accuracy:0.623656
iter 44000 train_accuracy:0.725550
iter 46000 train_accuracy:0.610343
iter 48000 train_accuracy:0.751152
Done training, using time:(min) 2.5652570009231566
Done training
[-24.72967924 -82.53314759  -0.45824203 ...  71.2698941   77.80467792
  83.31726411]
training resul

In [48]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=1, eta=0.01, tol=0.01,max_iter=20000)

Done kernel generating
iter 0 train_accuracy:0.520225


/home/y247xie/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


iter 2000 train_accuracy:0.519713
iter 4000 train_accuracy:0.519713
iter 6000 train_accuracy:0.519713
iter 8000 train_accuracy:0.521249
iter 10000 train_accuracy:0.522273
iter 12000 train_accuracy:0.522273
iter 14000 train_accuracy:0.520225
iter 16000 train_accuracy:0.521249
iter 18000 train_accuracy:0.522785
Done training, using time:(min) 1.0691288113594055
Done training
[-0.00228032 -0.16220508 -0.16201477 ...  0.18145629  0.13888216
  0.15907172]
training result
0.4987199180747568
test result
0.5165


In [29]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=0, eta=0.001, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.524322
iter 2000 train_accuracy:0.568356
iter 4000 train_accuracy:0.577573
iter 6000 train_accuracy:0.579109
iter 8000 train_accuracy:0.619048
iter 10000 train_accuracy:0.633385
iter 12000 train_accuracy:0.570405
iter 14000 train_accuracy:0.589350
iter 16000 train_accuracy:0.669739
iter 18000 train_accuracy:0.606759
iter 20000 train_accuracy:0.658986
iter 22000 train_accuracy:0.646697
iter 24000 train_accuracy:0.707117
iter 26000 train_accuracy:0.713262
iter 28000 train_accuracy:0.640553
iter 30000 train_accuracy:0.660522
iter 32000 train_accuracy:0.738863
iter 34000 train_accuracy:0.654378
iter 36000 train_accuracy:0.640041
iter 38000 train_accuracy:0.646185
iter 40000 train_accuracy:0.627752
iter 42000 train_accuracy:0.738351
iter 44000 train_accuracy:0.624680
iter 46000 train_accuracy:0.717870
iter 48000 train_accuracy:0.708141
Done training, using time:(min) 2.2449634432792664
Done training
[-2.55404882 -8.23323814 -0.05373824 ...  7.1

In [30]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=0, eta=0.0001, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.573989
iter 4000 train_accuracy:0.589350
iter 6000 train_accuracy:0.585765
iter 8000 train_accuracy:0.628776
iter 10000 train_accuracy:0.636969
iter 12000 train_accuracy:0.589350
iter 14000 train_accuracy:0.619048
iter 16000 train_accuracy:0.652330
iter 18000 train_accuracy:0.619048
iter 20000 train_accuracy:0.658986
iter 22000 train_accuracy:0.704045
iter 24000 train_accuracy:0.644649
iter 26000 train_accuracy:0.737327
iter 28000 train_accuracy:0.651818
iter 30000 train_accuracy:0.704557
iter 32000 train_accuracy:0.758833
iter 34000 train_accuracy:0.693804
iter 36000 train_accuracy:0.630824
iter 38000 train_accuracy:0.663082
iter 40000 train_accuracy:0.694828
iter 42000 train_accuracy:0.706605
iter 44000 train_accuracy:0.714286
iter 46000 train_accuracy:0.692780
iter 48000 train_accuracy:0.691244
Done training, using time:(min) 2.2375803788503013
Done training
[-0.25375142 -0.80916862 -0.01319075 ...  0.7

In [31]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=10, eta=0.001, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.520737
iter 4000 train_accuracy:0.520225
iter 6000 train_accuracy:0.519713
iter 8000 train_accuracy:0.519713
iter 10000 train_accuracy:0.521249
iter 12000 train_accuracy:0.522273
iter 14000 train_accuracy:0.520737
iter 16000 train_accuracy:0.519201
iter 18000 train_accuracy:0.519713
iter 20000 train_accuracy:0.520225
iter 22000 train_accuracy:0.519713
iter 24000 train_accuracy:0.520225
iter 26000 train_accuracy:0.520225
iter 28000 train_accuracy:0.521761
iter 30000 train_accuracy:0.521761
iter 32000 train_accuracy:0.519713
iter 34000 train_accuracy:0.520225
iter 36000 train_accuracy:0.522273
iter 38000 train_accuracy:0.521761
iter 40000 train_accuracy:0.519713
iter 42000 train_accuracy:0.520737
iter 44000 train_accuracy:0.520225
iter 46000 train_accuracy:0.521761
iter 48000 train_accuracy:0.520737
Done training, using time:(min) 2.2182202696800233
Done training
[-0.00049907 -0.01286958 -0.01324424 ...  0.0

In [32]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=30, eta=0.001, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.520225
iter 4000 train_accuracy:0.520225
iter 6000 train_accuracy:0.520225
iter 8000 train_accuracy:0.520225
iter 10000 train_accuracy:0.520225
iter 12000 train_accuracy:0.520225
iter 14000 train_accuracy:0.520225
iter 16000 train_accuracy:0.520225
iter 18000 train_accuracy:0.520225
iter 20000 train_accuracy:0.520225
iter 22000 train_accuracy:0.520225
iter 24000 train_accuracy:0.520225
iter 26000 train_accuracy:0.520225
iter 28000 train_accuracy:0.520225
iter 30000 train_accuracy:0.520225
iter 32000 train_accuracy:0.520225
iter 34000 train_accuracy:0.520225
iter 36000 train_accuracy:0.520225
iter 38000 train_accuracy:0.520225
iter 40000 train_accuracy:0.520225
iter 42000 train_accuracy:0.520225
iter 44000 train_accuracy:0.520225
iter 46000 train_accuracy:0.520225
iter 48000 train_accuracy:0.520225
Done training, using time:(min) 2.213572013378143
Done training
[-0.00146557 -0.00285996 -0.00867031 ...  0.00

In [33]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=1, eta=0.001, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.534562
iter 2000 train_accuracy:0.533026
iter 4000 train_accuracy:0.534562
iter 6000 train_accuracy:0.539171
iter 8000 train_accuracy:0.549411
iter 10000 train_accuracy:0.535074
iter 12000 train_accuracy:0.550435
iter 14000 train_accuracy:0.534562
iter 16000 train_accuracy:0.545827
iter 18000 train_accuracy:0.554019
iter 20000 train_accuracy:0.542755
iter 22000 train_accuracy:0.554019
iter 24000 train_accuracy:0.538658
iter 26000 train_accuracy:0.536610
iter 28000 train_accuracy:0.541219
iter 30000 train_accuracy:0.551971
iter 32000 train_accuracy:0.562724
iter 34000 train_accuracy:0.537122
iter 36000 train_accuracy:0.540195
iter 38000 train_accuracy:0.545827
iter 40000 train_accuracy:0.542243
iter 42000 train_accuracy:0.545315
iter 44000 train_accuracy:0.560164
iter 46000 train_accuracy:0.550947
iter 48000 train_accuracy:0.537634
Done training, using time:(min) 2.1733374913533527
Done training
[-0.15437237 -0.15523217 -0.006991   ...  0.1

In [44]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=0.1, eta=0.01, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.576037
iter 4000 train_accuracy:0.551459
iter 6000 train_accuracy:0.551971
iter 8000 train_accuracy:0.548899
iter 10000 train_accuracy:0.582181
iter 12000 train_accuracy:0.575013
iter 14000 train_accuracy:0.603687
iter 16000 train_accuracy:0.577573
iter 18000 train_accuracy:0.594982
iter 20000 train_accuracy:0.563748
iter 22000 train_accuracy:0.553507
iter 24000 train_accuracy:0.563748
iter 26000 train_accuracy:0.573477
iter 28000 train_accuracy:0.624680
iter 30000 train_accuracy:0.597542
iter 32000 train_accuracy:0.609319
iter 34000 train_accuracy:0.634409
iter 36000 train_accuracy:0.612391
iter 38000 train_accuracy:0.645673
iter 40000 train_accuracy:0.581157
iter 42000 train_accuracy:0.577573
iter 44000 train_accuracy:0.572965
iter 46000 train_accuracy:0.613415
iter 48000 train_accuracy:0.547363
Done training, using time:(min) 2.2342421571413675
Done training
[-1.07524982e+00 -1.67430242e+00 -3.39500117e

In [45]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=0.3, eta=0.01, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.520225


/home/y247xie/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


iter 2000 train_accuracy:0.537122
iter 4000 train_accuracy:0.524834
iter 6000 train_accuracy:0.536610
iter 8000 train_accuracy:0.534562
iter 10000 train_accuracy:0.529442
iter 12000 train_accuracy:0.532002
iter 14000 train_accuracy:0.529954
iter 16000 train_accuracy:0.537634
iter 18000 train_accuracy:0.535586
iter 20000 train_accuracy:0.538658
iter 22000 train_accuracy:0.532002
iter 24000 train_accuracy:0.529954
iter 26000 train_accuracy:0.538658
iter 28000 train_accuracy:0.532002
iter 30000 train_accuracy:0.525858
iter 32000 train_accuracy:0.529442
iter 34000 train_accuracy:0.531490
iter 36000 train_accuracy:0.525346
iter 38000 train_accuracy:0.533538
iter 40000 train_accuracy:0.533538
iter 42000 train_accuracy:0.536610
iter 44000 train_accuracy:0.525346
iter 46000 train_accuracy:0.535074
iter 48000 train_accuracy:0.529954
Done training, using time:(min) 2.690360891819
Done training
[-0.22515619 -0.55284273 -0.38739411 ...  0.57308675  0.60670647
  0.53357378]
training result
0.505888

In [46]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=0.01, eta=0.01, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.520225


/home/y247xie/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


iter 2000 train_accuracy:0.556068
iter 4000 train_accuracy:0.562212
iter 6000 train_accuracy:0.591398
iter 8000 train_accuracy:0.626216
iter 10000 train_accuracy:0.592422
iter 12000 train_accuracy:0.587302
iter 14000 train_accuracy:0.592422
iter 16000 train_accuracy:0.616999
iter 18000 train_accuracy:0.603687
iter 20000 train_accuracy:0.603175
iter 22000 train_accuracy:0.565284
iter 24000 train_accuracy:0.625192
iter 26000 train_accuracy:0.617512
iter 28000 train_accuracy:0.595494
iter 30000 train_accuracy:0.602151
iter 32000 train_accuracy:0.595494
iter 34000 train_accuracy:0.586278
iter 36000 train_accuracy:0.598566
iter 38000 train_accuracy:0.572965
iter 40000 train_accuracy:0.614439
iter 42000 train_accuracy:0.623656
iter 44000 train_accuracy:0.603175
iter 46000 train_accuracy:0.577061
iter 48000 train_accuracy:0.571429
Done training, using time:(min) 2.6066909591356913
Done training
[-1.08921901e+01 -1.64344452e+01 -3.47382476e-03 ...  1.60878272e+01
  1.61031817e+01  1.65576711e+

# gaussian kernel exps

In [40]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0.1, eta=0.01, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.601639
iter 4000 train_accuracy:0.612391
iter 6000 train_accuracy:0.596006
iter 8000 train_accuracy:0.603175
iter 10000 train_accuracy:0.590886
iter 12000 train_accuracy:0.603175
iter 14000 train_accuracy:0.556580
iter 16000 train_accuracy:0.567332
iter 18000 train_accuracy:0.611879
iter 20000 train_accuracy:0.562724
iter 22000 train_accuracy:0.605223
iter 24000 train_accuracy:0.608295
iter 26000 train_accuracy:0.600614
iter 28000 train_accuracy:0.582181
iter 30000 train_accuracy:0.604711
iter 32000 train_accuracy:0.605223
iter 34000 train_accuracy:0.612903
iter 36000 train_accuracy:0.610343
iter 38000 train_accuracy:0.608295
iter 40000 train_accuracy:0.596518
iter 42000 train_accuracy:0.612391
iter 44000 train_accuracy:0.603687
iter 46000 train_accuracy:0.605223
iter 48000 train_accuracy:0.601126
Done training, using time:(min) 2.2019865552584332
Done training
[-1.21471422 -1.67036781 -1.34451458 ...  1.8

In [49]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0.3, eta=0.001, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.585253
iter 4000 train_accuracy:0.589350
iter 6000 train_accuracy:0.600614
iter 8000 train_accuracy:0.589350
iter 10000 train_accuracy:0.594470
iter 12000 train_accuracy:0.593958
iter 14000 train_accuracy:0.593958
iter 16000 train_accuracy:0.593958
iter 18000 train_accuracy:0.604199
iter 20000 train_accuracy:0.592934
iter 22000 train_accuracy:0.595494
iter 24000 train_accuracy:0.593446
iter 26000 train_accuracy:0.603175
iter 28000 train_accuracy:0.592934
iter 30000 train_accuracy:0.597542
iter 32000 train_accuracy:0.596006
iter 34000 train_accuracy:0.594470
iter 36000 train_accuracy:0.596006
iter 38000 train_accuracy:0.594470
iter 40000 train_accuracy:0.591398
iter 42000 train_accuracy:0.594982
iter 44000 train_accuracy:0.593446
iter 46000 train_accuracy:0.594470
iter 48000 train_accuracy:0.598054
Done training, using time:(min) 2.2161025166511537
Done training
[-0.43570644 -0.56634806 -0.46884925 ...  0.6

In [35]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=1, eta=0.001, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.556580
iter 4000 train_accuracy:0.564772
iter 6000 train_accuracy:0.585253
iter 8000 train_accuracy:0.573989
iter 10000 train_accuracy:0.582693
iter 12000 train_accuracy:0.565284
iter 14000 train_accuracy:0.557092
iter 16000 train_accuracy:0.579621
iter 18000 train_accuracy:0.568868
iter 20000 train_accuracy:0.564772
iter 22000 train_accuracy:0.571941
iter 24000 train_accuracy:0.562212
iter 26000 train_accuracy:0.572453
iter 28000 train_accuracy:0.584229
iter 30000 train_accuracy:0.569892
iter 32000 train_accuracy:0.570917
iter 34000 train_accuracy:0.556580
iter 36000 train_accuracy:0.566820
iter 38000 train_accuracy:0.563236
iter 40000 train_accuracy:0.585253
iter 42000 train_accuracy:0.571941
iter 44000 train_accuracy:0.565284
iter 46000 train_accuracy:0.554019
iter 48000 train_accuracy:0.555556
Done training, using time:(min) 2.2260678807894387
Done training
[-0.14191776 -0.16183555 -0.15345055 ...  0.1

In [36]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=10, eta=0.001, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.479775
iter 2000 train_accuracy:0.520225
iter 4000 train_accuracy:0.520225
iter 6000 train_accuracy:0.520225
iter 8000 train_accuracy:0.520225
iter 10000 train_accuracy:0.520225
iter 12000 train_accuracy:0.520225
iter 14000 train_accuracy:0.520225
iter 16000 train_accuracy:0.520225
iter 18000 train_accuracy:0.520225
iter 20000 train_accuracy:0.520225
iter 22000 train_accuracy:0.520225
iter 24000 train_accuracy:0.520225
iter 26000 train_accuracy:0.520225
iter 28000 train_accuracy:0.520225
iter 30000 train_accuracy:0.520225
iter 32000 train_accuracy:0.520225
iter 34000 train_accuracy:0.520225
iter 36000 train_accuracy:0.520225
iter 38000 train_accuracy:0.520225
iter 40000 train_accuracy:0.520225
iter 42000 train_accuracy:0.520225
iter 44000 train_accuracy:0.520225
iter 46000 train_accuracy:0.520225
iter 48000 train_accuracy:0.520225
Done training, using time:(min) 2.260877780119578
Done training
[-0.01481768 -0.01459545 -0.01419502 ...  0.01

In [34]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0, eta=0.001, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.585765
iter 4000 train_accuracy:0.602151
iter 6000 train_accuracy:0.602663
iter 8000 train_accuracy:0.598054
iter 10000 train_accuracy:0.603175
iter 12000 train_accuracy:0.606247
iter 14000 train_accuracy:0.605735
iter 16000 train_accuracy:0.610855
iter 18000 train_accuracy:0.612391
iter 20000 train_accuracy:0.612903
iter 22000 train_accuracy:0.617512
iter 24000 train_accuracy:0.616999
iter 26000 train_accuracy:0.618536
iter 28000 train_accuracy:0.622632
iter 30000 train_accuracy:0.625192
iter 32000 train_accuracy:0.623656
iter 34000 train_accuracy:0.622632
iter 36000 train_accuracy:0.626216
iter 38000 train_accuracy:0.627240
iter 40000 train_accuracy:0.629288
iter 42000 train_accuracy:0.633385
iter 44000 train_accuracy:0.634409
iter 46000 train_accuracy:0.632873
iter 48000 train_accuracy:0.634409
Done training, using time:(min) 2.194558862845103
Done training
[-5.4906544  -8.88665359 -5.54955064 ...  9.62

In [41]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0.3, eta=0.01, tol=0.01)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.600614
iter 4000 train_accuracy:0.583205
iter 6000 train_accuracy:0.559652
iter 8000 train_accuracy:0.594982
iter 10000 train_accuracy:0.536098
iter 12000 train_accuracy:0.595494
iter 14000 train_accuracy:0.563236
iter 16000 train_accuracy:0.575013
iter 18000 train_accuracy:0.602151
iter 20000 train_accuracy:0.565284
iter 22000 train_accuracy:0.596006
iter 24000 train_accuracy:0.576037
iter 26000 train_accuracy:0.578085
iter 28000 train_accuracy:0.559652
iter 30000 train_accuracy:0.603175
iter 32000 train_accuracy:0.554019
iter 34000 train_accuracy:0.602151
iter 36000 train_accuracy:0.569892
iter 38000 train_accuracy:0.573477
iter 40000 train_accuracy:0.589862
iter 42000 train_accuracy:0.540707
iter 44000 train_accuracy:0.573989
iter 46000 train_accuracy:0.589350
iter 48000 train_accuracy:0.590374
Done training, using time:(min) 2.2309987624486287
Done training
[-0.47322899 -0.58491732 -0.4929542  ...  0.6

In [47]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0, eta=0.001, tol=0.1,max_iter=500000)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.587814
iter 4000 train_accuracy:0.600102
iter 6000 train_accuracy:0.603687
iter 8000 train_accuracy:0.602663
iter 10000 train_accuracy:0.600614
iter 12000 train_accuracy:0.609319
iter 14000 train_accuracy:0.609319
iter 16000 train_accuracy:0.609831
iter 18000 train_accuracy:0.611367
iter 20000 train_accuracy:0.616999
iter 22000 train_accuracy:0.621096
iter 24000 train_accuracy:0.618536
iter 26000 train_accuracy:0.618024
iter 28000 train_accuracy:0.624168
iter 30000 train_accuracy:0.627240
iter 32000 train_accuracy:0.621096
iter 34000 train_accuracy:0.628264
iter 36000 train_accuracy:0.628264
iter 38000 train_accuracy:0.627752
iter 40000 train_accuracy:0.630312
iter 42000 train_accuracy:0.632873
iter 44000 train_accuracy:0.630312
iter 46000 train_accuracy:0.633897
iter 48000 train_accuracy:0.634409
iter 50000 train_accuracy:0.636457
iter 52000 train_accuracy:0.636969
iter 54000 train_accuracy:0.637481
iter 

iter 458000 train_accuracy:0.702509
iter 460000 train_accuracy:0.705069
iter 462000 train_accuracy:0.705581
iter 464000 train_accuracy:0.707629
iter 466000 train_accuracy:0.703533
iter 468000 train_accuracy:0.704557
iter 470000 train_accuracy:0.704557
iter 472000 train_accuracy:0.707629
iter 474000 train_accuracy:0.705069
iter 476000 train_accuracy:0.704557
iter 478000 train_accuracy:0.707117
iter 480000 train_accuracy:0.706605
iter 482000 train_accuracy:0.706605
iter 484000 train_accuracy:0.707117
iter 486000 train_accuracy:0.706605
iter 488000 train_accuracy:0.706605
iter 490000 train_accuracy:0.703533
iter 492000 train_accuracy:0.705581
iter 494000 train_accuracy:0.706605
iter 496000 train_accuracy:0.710701
iter 498000 train_accuracy:0.708141
Done training, using time:(min) 22.358522486686706
Done training
[-51.77448543 -85.54993795 -35.28667167 ...  92.3613971   76.3212428
  87.38571095]
training result
0.7081413210445469
test result
0.599


In [42]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0.01, eta=0.001, tol=0.01)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.589862
iter 4000 train_accuracy:0.599590
iter 6000 train_accuracy:0.601639
iter 8000 train_accuracy:0.602151
iter 10000 train_accuracy:0.601639
iter 12000 train_accuracy:0.603687
iter 14000 train_accuracy:0.609319
iter 16000 train_accuracy:0.610343
iter 18000 train_accuracy:0.610855
iter 20000 train_accuracy:0.612903
iter 22000 train_accuracy:0.617512
iter 24000 train_accuracy:0.615975
iter 26000 train_accuracy:0.618536
iter 28000 train_accuracy:0.619048
iter 30000 train_accuracy:0.619560
iter 32000 train_accuracy:0.618536
iter 34000 train_accuracy:0.619048
iter 36000 train_accuracy:0.618536
iter 38000 train_accuracy:0.624680
iter 40000 train_accuracy:0.620072
iter 42000 train_accuracy:0.627240
iter 44000 train_accuracy:0.626216
iter 46000 train_accuracy:0.627752
iter 48000 train_accuracy:0.629288
Done training, using time:(min) 2.177192775408427
Done training
[-4.40544097 -7.00185946 -4.49301202 ...  7.57

In [43]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0, eta=0.0001, tol=0.01)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.520225
iter 4000 train_accuracy:0.527906
iter 6000 train_accuracy:0.548899
iter 8000 train_accuracy:0.562724
iter 10000 train_accuracy:0.573989
iter 12000 train_accuracy:0.580645
iter 14000 train_accuracy:0.583717
iter 16000 train_accuracy:0.583717
iter 18000 train_accuracy:0.586278
iter 20000 train_accuracy:0.585765
iter 22000 train_accuracy:0.589350
iter 24000 train_accuracy:0.588326
iter 26000 train_accuracy:0.590374
iter 28000 train_accuracy:0.591398
iter 30000 train_accuracy:0.592934
iter 32000 train_accuracy:0.595494
iter 34000 train_accuracy:0.596518
iter 36000 train_accuracy:0.594470
iter 38000 train_accuracy:0.600102
iter 40000 train_accuracy:0.603175
iter 42000 train_accuracy:0.601639
iter 44000 train_accuracy:0.603687
iter 46000 train_accuracy:0.601126
iter 48000 train_accuracy:0.603175
Done training, using time:(min) 2.219695496559143
Done training
[-0.66279991 -0.82804417 -0.72267349 ...  0.90

In [52]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0, eta=1, tol=0.001)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.529442
iter 4000 train_accuracy:0.542243
iter 6000 train_accuracy:0.545827
iter 8000 train_accuracy:0.546339
iter 10000 train_accuracy:0.549923
iter 12000 train_accuracy:0.546851
iter 14000 train_accuracy:0.563236
iter 16000 train_accuracy:0.557092
iter 18000 train_accuracy:0.567844
iter 20000 train_accuracy:0.549923
iter 22000 train_accuracy:0.570405
iter 24000 train_accuracy:0.546339
iter 26000 train_accuracy:0.576549
iter 28000 train_accuracy:0.552483
iter 30000 train_accuracy:0.572965
iter 32000 train_accuracy:0.567844
iter 34000 train_accuracy:0.558628
iter 36000 train_accuracy:0.558628
iter 38000 train_accuracy:0.560164
iter 40000 train_accuracy:0.607783
iter 42000 train_accuracy:0.575525
iter 44000 train_accuracy:0.554019
iter 46000 train_accuracy:0.588838
iter 48000 train_accuracy:0.564260
Done training, using time:(min) 2.206505878766378
Done training
[-7002.24018911 -8274.49835721  -695.81322716 

In [53]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0.03, eta=0.0001, tol=0.01)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.520737
iter 4000 train_accuracy:0.529954
iter 6000 train_accuracy:0.556580
iter 8000 train_accuracy:0.563748
iter 10000 train_accuracy:0.568868
iter 12000 train_accuracy:0.580645
iter 14000 train_accuracy:0.582181
iter 16000 train_accuracy:0.581669
iter 18000 train_accuracy:0.586278
iter 20000 train_accuracy:0.590886
iter 22000 train_accuracy:0.588838
iter 24000 train_accuracy:0.589350
iter 26000 train_accuracy:0.592422
iter 28000 train_accuracy:0.590886
iter 30000 train_accuracy:0.592934
iter 32000 train_accuracy:0.597542
iter 34000 train_accuracy:0.595494
iter 36000 train_accuracy:0.594982
iter 38000 train_accuracy:0.597542
iter 40000 train_accuracy:0.600102
iter 42000 train_accuracy:0.600102
iter 44000 train_accuracy:0.599590
iter 46000 train_accuracy:0.602151
iter 48000 train_accuracy:0.604711
Done training, using time:(min) 2.2074294567108153
Done training
[-0.60310069 -0.77834608 -0.65930034 ...  0.8

In [54]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0.01, eta=0.0001, tol=0.01, batch_size=500)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.520225
iter 4000 train_accuracy:0.525858
iter 6000 train_accuracy:0.542243
iter 8000 train_accuracy:0.547875
iter 10000 train_accuracy:0.560676
iter 12000 train_accuracy:0.573477
iter 14000 train_accuracy:0.581669
iter 16000 train_accuracy:0.580133
iter 18000 train_accuracy:0.582181
iter 20000 train_accuracy:0.585765
iter 22000 train_accuracy:0.586278
iter 24000 train_accuracy:0.586278
iter 26000 train_accuracy:0.589350
iter 28000 train_accuracy:0.587814
iter 30000 train_accuracy:0.589862
iter 32000 train_accuracy:0.589350
iter 34000 train_accuracy:0.590374
iter 36000 train_accuracy:0.591910
iter 38000 train_accuracy:0.594470
iter 40000 train_accuracy:0.593958
iter 42000 train_accuracy:0.596518
iter 44000 train_accuracy:0.596518
iter 46000 train_accuracy:0.597030
iter 48000 train_accuracy:0.597030
Done training, using time:(min) 1.7432286302248636
Done training
[-0.50969757 -0.6309732  -0.55838775 ...  0.6

In [55]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0.01, eta=0.0001, tol=0.01, batch_size=50)

Done kernel generating
iter 0 train_accuracy:0.520225
iter 2000 train_accuracy:0.520225
iter 4000 train_accuracy:0.520225
iter 6000 train_accuracy:0.520225
iter 8000 train_accuracy:0.520225
iter 10000 train_accuracy:0.520225
iter 12000 train_accuracy:0.520225
iter 14000 train_accuracy:0.520225
iter 16000 train_accuracy:0.520225
iter 18000 train_accuracy:0.520225
iter 20000 train_accuracy:0.520225
iter 22000 train_accuracy:0.520225
iter 24000 train_accuracy:0.520225
iter 26000 train_accuracy:0.520225
iter 28000 train_accuracy:0.520737
iter 30000 train_accuracy:0.520225
iter 32000 train_accuracy:0.520737
iter 34000 train_accuracy:0.522273
iter 36000 train_accuracy:0.522785
iter 38000 train_accuracy:0.522273
iter 40000 train_accuracy:0.522785
iter 42000 train_accuracy:0.527394
iter 44000 train_accuracy:0.522785
iter 46000 train_accuracy:0.527394
iter 48000 train_accuracy:0.529442
Done training, using time:(min) 0.23624784549077352
Done training
[-0.0611975  -0.06100673 -0.05920965 ...  0.

In [56]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0.1, eta=0.001, tol=0.1,batch_size=50, max_iter=500000)

Done kernel generating
iter 0 train_accuracy:0.479775
iter 2000 train_accuracy:0.520225
iter 4000 train_accuracy:0.521249
iter 6000 train_accuracy:0.520737
iter 8000 train_accuracy:0.584229
iter 10000 train_accuracy:0.549411
iter 12000 train_accuracy:0.528418
iter 14000 train_accuracy:0.538146
iter 16000 train_accuracy:0.541219
iter 18000 train_accuracy:0.555044
iter 20000 train_accuracy:0.550947
iter 22000 train_accuracy:0.550947
iter 24000 train_accuracy:0.548899
iter 26000 train_accuracy:0.561188
iter 28000 train_accuracy:0.554019
iter 30000 train_accuracy:0.564772
iter 32000 train_accuracy:0.582693
iter 34000 train_accuracy:0.552995
iter 36000 train_accuracy:0.553507
iter 38000 train_accuracy:0.551459
iter 40000 train_accuracy:0.570917
iter 42000 train_accuracy:0.583205
iter 44000 train_accuracy:0.565284
iter 46000 train_accuracy:0.548899
iter 48000 train_accuracy:0.563748
iter 50000 train_accuracy:0.564772
iter 52000 train_accuracy:0.549923
iter 54000 train_accuracy:0.553507
iter 

iter 458000 train_accuracy:0.548387
iter 460000 train_accuracy:0.566308
iter 462000 train_accuracy:0.544291
iter 464000 train_accuracy:0.557092
iter 466000 train_accuracy:0.584229
iter 468000 train_accuracy:0.564772
iter 470000 train_accuracy:0.565284
iter 472000 train_accuracy:0.551971
iter 474000 train_accuracy:0.565284
iter 476000 train_accuracy:0.562724
iter 478000 train_accuracy:0.549923
iter 480000 train_accuracy:0.538658
iter 482000 train_accuracy:0.579621
iter 484000 train_accuracy:0.565796
iter 486000 train_accuracy:0.582693
iter 488000 train_accuracy:0.584741
iter 490000 train_accuracy:0.548899
iter 492000 train_accuracy:0.551459
iter 494000 train_accuracy:0.543779
iter 496000 train_accuracy:0.565284
iter 498000 train_accuracy:0.540707
Done training, using time:(min) 2.358226732412974
Done training
[-0.11318626 -0.12767079 -0.11996031 ...  0.13870435  0.13704088
  0.13794387]
training result
0.5627240143369175
test result
0.5585


# inhomogeneous kernel exps

In [37]:
experiment(inhomogeneous_poly_kernel, X, y, X_test, y_test, regu_para=0, eta=0.001, tol=0.01)

Done kernel generating
iter 0 train_accuracy:0.525858


/home/y247xie/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


iter 2000 train_accuracy:0.962110
iter 4000 train_accuracy:0.988223
Done training, using time:(min) 0.28302624622980754
Done training
[-0.044  -0.2965 -0.127  ...  0.266   0.1635  0.26  ]
training result
0.9912954429083461
test result
0.6015


In [38]:
experiment(inhomogeneous_poly_kernel, X, y, X_test, y_test, regu_para=1, eta=0.001, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.518689


/home/y247xie/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


iter 2000 train_accuracy:0.544291
iter 4000 train_accuracy:0.570405
iter 6000 train_accuracy:0.568356
iter 8000 train_accuracy:0.594982
iter 10000 train_accuracy:0.586790
iter 12000 train_accuracy:0.699949
iter 14000 train_accuracy:0.648746
iter 16000 train_accuracy:0.830005
iter 18000 train_accuracy:0.526370
iter 20000 train_accuracy:0.644137
iter 22000 train_accuracy:0.573477
iter 24000 train_accuracy:0.578085
iter 26000 train_accuracy:0.811572
iter 28000 train_accuracy:0.658986
iter 30000 train_accuracy:0.653354
iter 32000 train_accuracy:0.796723
iter 34000 train_accuracy:0.813620
iter 36000 train_accuracy:0.734767
iter 38000 train_accuracy:0.712750
iter 40000 train_accuracy:0.547363
iter 42000 train_accuracy:0.539171
iter 44000 train_accuracy:0.631848
iter 46000 train_accuracy:0.687660
iter 48000 train_accuracy:0.830517
Done training, using time:(min) 2.8351170420646667
Done training
[-0.02731652 -0.18705636 -0.09454211 ...  0.16458566  0.12203252
  0.17388478]
training result
0.63

In [39]:
experiment(inhomogeneous_poly_kernel, X, y, X_test, y_test, regu_para=10, eta=0.01, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.561700


/home/y247xie/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


iter 2000 train_accuracy:0.521249
iter 4000 train_accuracy:0.524322
iter 6000 train_accuracy:0.520737
iter 8000 train_accuracy:0.529954
iter 10000 train_accuracy:0.522273
iter 12000 train_accuracy:0.521761
iter 14000 train_accuracy:0.503328
iter 16000 train_accuracy:0.479775
iter 18000 train_accuracy:0.519713
iter 20000 train_accuracy:0.521761
iter 22000 train_accuracy:0.521761
iter 24000 train_accuracy:0.532002
iter 26000 train_accuracy:0.522273
iter 28000 train_accuracy:0.494112
iter 30000 train_accuracy:0.521249
iter 32000 train_accuracy:0.521761
iter 34000 train_accuracy:0.483359
iter 36000 train_accuracy:0.519201
iter 38000 train_accuracy:0.520225
iter 40000 train_accuracy:0.522273
iter 42000 train_accuracy:0.516129
iter 44000 train_accuracy:0.494112
iter 46000 train_accuracy:0.492063
iter 48000 train_accuracy:0.519713
Done training, using time:(min) 2.832144820690155
Done training
[-0.00625517 -0.0331439  -0.01440623 ...  0.01616075  0.02451547
  0.018909  ]
training result
0.520

In [50]:
experiment(inhomogeneous_poly_kernel, X, y, X_test, y_test, regu_para=10, eta=0.001, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.567332


/home/y247xie/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


iter 2000 train_accuracy:0.595494
iter 4000 train_accuracy:0.573989
iter 6000 train_accuracy:0.536610
iter 8000 train_accuracy:0.573989
iter 10000 train_accuracy:0.535074
iter 12000 train_accuracy:0.548899
iter 14000 train_accuracy:0.542243
iter 16000 train_accuracy:0.593446
iter 18000 train_accuracy:0.527906
iter 20000 train_accuracy:0.556068
iter 22000 train_accuracy:0.538658
iter 24000 train_accuracy:0.580645
iter 26000 train_accuracy:0.589862
iter 28000 train_accuracy:0.602151
iter 30000 train_accuracy:0.561188
iter 32000 train_accuracy:0.541731
iter 34000 train_accuracy:0.537122
iter 36000 train_accuracy:0.548899
iter 38000 train_accuracy:0.573477
iter 40000 train_accuracy:0.550947
iter 42000 train_accuracy:0.523297
iter 44000 train_accuracy:0.497184
iter 46000 train_accuracy:0.533026
iter 48000 train_accuracy:0.536098
Done training, using time:(min) 2.794492701689402
Done training
[-0.00029785 -0.02198898 -0.01531829 ...  0.01808216  0.01348821
  0.01795119]
training result
0.548

In [51]:
experiment(inhomogeneous_poly_kernel, X, y, X_test, y_test, regu_para=1, eta=0.03, tol=0.1)

Done kernel generating
iter 0 train_accuracy:0.562212


/home/y247xie/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


iter 2000 train_accuracy:0.483359
iter 4000 train_accuracy:0.538146
iter 6000 train_accuracy:0.572965
iter 8000 train_accuracy:0.556068
iter 10000 train_accuracy:0.548387
iter 12000 train_accuracy:0.569892
iter 14000 train_accuracy:0.483871
iter 16000 train_accuracy:0.543779
iter 18000 train_accuracy:0.493600
iter 20000 train_accuracy:0.543779
iter 22000 train_accuracy:0.575525
iter 24000 train_accuracy:0.510497
iter 26000 train_accuracy:0.530466
iter 28000 train_accuracy:0.485407
iter 30000 train_accuracy:0.547363
iter 32000 train_accuracy:0.524322
iter 34000 train_accuracy:0.513569
iter 36000 train_accuracy:0.533026
iter 38000 train_accuracy:0.558116
iter 40000 train_accuracy:0.479775
iter 42000 train_accuracy:0.487967
iter 44000 train_accuracy:0.523810
iter 46000 train_accuracy:0.479775
iter 48000 train_accuracy:0.483359
Done training, using time:(min) 2.787503182888031
Done training
[-0.03852538 -0.12266537 -0.24466277 ...  0.20037286  0.21487781
  0.20457185]
training result
0.481